### 导入数据

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

train = pd.read_csv('data/train.csv') # 读取数据
test = pd.read_csv('data/test.csv')
test1 = pd.read_csv('data/test.csv')

### 观察数据

In [2]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [4]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


### 数据预处理

1、观察数据后发现，可去除PassengerId，Name，Ticket（无关）；Cabin（缺失过多）几列特征

In [5]:
train = train.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


2、至此Age字段和Embarked字段有缺失，需补充

In [6]:
train['Age'] = train['Age'].fillna(train['Age'].median()) # 由于Age最大值最小值差距过大，这里用中位数进行填充

In [7]:
train['Embarked'].value_counts() # 统计Embarked各类别出现次数

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [8]:
train['Embarked'] = train['Embarked'].fillna('S') # 用出现最多的类别填充Embarked列
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null object
Age         891 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
Embarked    891 non-null object
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


3、将Sex和Embarked由字符串转化为数字

In [9]:
from sklearn.preprocessing import LabelEncoder

train['Sex'] = LabelEncoder().fit_transform(train['Sex'])
train['Embarked'] = LabelEncoder().fit_transform(train['Embarked'])

train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null int64
Age         891 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
Embarked    891 non-null int64
dtypes: float64(2), int64(6)
memory usage: 55.8 KB


4、将数据做标准化处理

In [10]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler()
for feature in train.columns:
    if feature != 'Survived':
        train[feature] = scaler.fit_transform(train[feature].values.reshape(-1, 1)) # 标准化处理数据

train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,0.827377,0.737695,-0.565736,0.432793,-0.473674,-0.502445,0.585954
1,1,-1.566107,-1.355574,0.663861,0.432793,-0.473674,0.786845,-1.942303
2,1,0.827377,-1.355574,-0.258337,-0.474545,-0.473674,-0.488854,0.585954
3,1,-1.566107,-1.355574,0.433312,0.432793,-0.473674,0.420730,0.585954
4,0,0.827377,0.737695,0.433312,-0.474545,-0.473674,-0.486337,0.585954


### 使用逻辑回归预测

In [11]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

x_train = train.drop(['Survived'], axis=1)
y_train = train['Survived']

C_params = [0.01, 0.1, 1, 10, 100] # 参数C,惩罚力度

for C_param in C_params:
    print('========================================')
    print('C_param = ', C_param)
    lr =  LogisticRegression(C=C_param, penalty='l1') # 正则化项采用l1范数
    score = cross_val_score(lr, x_train, y_train, cv=5) # 采用k折交叉验证，cv为折树
    print('score：',score.mean()) # 当前参数下交叉验证模型评分

C_param =  0.01
score： 0.7867150249291879
C_param =  0.1
score： 0.7834072218930012
C_param =  1
score： 0.7856923590327922
C_param =  10
score： 0.7856671088986509
C_param =  100
score： 0.7856671088986509


可见使用C=0.1时分数最高，score为0.786

### 使用随机森林预测

In [12]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

dt =RandomForestClassifier(n_estimators = 100,min_samples_split=4,min_samples_leaf=2)
score = cross_val_score(dt,x_train, y_train, cv=10)
print('score：',score.mean())

score： 0.8351395982294859


可见使用随机森林预测后准确率大大增加

In [13]:
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

test = test.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
test['Age'] = test['Age'].fillna(test['Age'].median())
test['Fare'] = test['Fare'].fillna(test['Fare'].median())
test['Embarked'] = test['Embarked'].fillna('S')

test['Sex'] = LabelEncoder().fit_transform(test['Sex'])
test['Embarked'] = LabelEncoder().fit_transform(test['Embarked'])

scaler = preprocessing.StandardScaler()
for feature in train.columns:
    if feature != 'Survived':
        test[feature] = scaler.fit_transform(test[feature].values.reshape(-1, 1)) # 标准化处理数据

test.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0.873482,0.755929,0.386231,-0.499470,-0.400248,-0.497413,-0.470915
1,0.873482,-1.322876,1.371370,0.616992,-0.400248,-0.512278,0.700767
2,-0.315819,0.755929,2.553537,-0.499470,-0.400248,-0.464100,-0.470915
3,0.873482,0.755929,-0.204852,-0.499470,-0.400248,-0.482475,0.700767
4,0.873482,-1.322876,-0.598908,0.616992,0.619896,-0.417492,0.700767


In [14]:
test['Fare'] = test['Fare'].fillna(test['Fare'].median())

dt.fit(x_train, y_train)
predict_data = dt.predict(test)

submission = pd.DataFrame({
    'PassengerId':test1['PassengerId'],
    'Survived':predict_data
})
submission.to_csv('gender_submission.csv', index=False)

提交之后分数为0.75598，有些糟糕